# Fine-Tuning T5 Model

## Pip & Import

In [1]:
!pip install datasets evaluate transformers rouge-score nltk
!apt install git-lfs
!pip install datasets
!pip install evaluate
!pip install accelerate>=0.20.1
!pip install rouge_score
!pip install transformers[torch]
!pip install sentencepiece
!pip install accelerate -U
!pip show transformers
!pip show accelerate
!pip install transformers
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.2 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=f874b4345d4a75f99799e1aabc5c0c359ab97c53459c1b544f74c85911f59f04
  Stored in directory: /root/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge-score
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 15 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 13.3 MB/s eta 0:00:00
Name: transformers
Version: 4.35.

In [2]:
import nltk
from datasets import load_dataset
from evaluate import load
from google.colab import drive
import nltk
import numpy as np
import transformers
import transformers
import datasets
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
import random
import pandas as pd
from transformers import AutoTokenizer
from IPython.display import display, HTML
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import AutoModelForSeq2SeqLM, AutoConfig
drive.mount('/content/drive')
nltk.download('punkt')
from transformers.utils import send_example_telemetry

Mounted at /content/drive


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


I Need this to access into my dataset on my account

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
import transformers

print(transformers.__version__)

4.35.2


In [5]:
send_example_telemetry("summarization_notebook", framework="pytorch")

## invoke model from HuggingFace

In [6]:
#model
model_checkpoint = "malmarjeh/t5-arabic-text-summarization"

## Loading the dataset

load my data from Hugging face

In [7]:
raw_datasets = load_dataset('RahafLog/T5_dataset')
metric = load("rouge")

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

In [8]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'answer'],
        num_rows: 24
    })
    validation: Dataset({
        features: ['id', 'text', 'answer'],
        num_rows: 7
    })
})

In [9]:
raw_datasets["train"][0]

{'id': 78496648,
 'text': 'الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:فلدى الدائرة التجارية الواحد والعشرون وباءً على القضية رقم ٦٩١ لعام ١٤٤٢ هـالمقامة من/ شركة ج.إ.ل.ا سجل تجاري (...)ضد/ شركة أ.ن.ا.ش.ش.و.غ ذلك (...) القاضي عبدالرحمن ب عبدﷲ ب سلامه رالقاضي حسام ب محمد ب علي العثيم عالقاضي هشام ب عبدﷲ ب سليمان المطرودي ع (الوقائع)تتلخص وقائع هذه القضية بالقدر اللازم للحكم فيها بأنه تقدم إلى المحكمة التجارية بالرياض المدعي/ وكالة: سعد مسعد ضيف ﷲ المطيري هوية رقم: (...) بموجب وكالة رقم: (٤٢٩٥٨٤٠) بلائحة دعوى ضد المدعى عليها مبيناً فيها بأن موكلته عملت على صيانة الأجهزة الطبية لصالح المدعى عليها من منتصف عام ٢٠١٦م إلى عام ٢٠١٩م وقد بلغ إجمالي قيمة عمل موكلته لدى المدعى عليها مبلغ وقدرة (٢.٦٥٢.٧٠٩) مليونان وستمائة واثنان وخمسون ألفاً وسبعمائة وتسعة ريالات، يوجد منها مبلغ وقدرة (٧٢٨.٩٣١.٣١) سبعمائة وتسعة الاف وواحد وثلاثون ريال عليها مطابقة رصيد، والمبلغ المتبقي يوجد عليه فواتير شراء والبالغة (١.٩٢٣.٧٧٨) مليون وتسعمائة وثلاث وعشرون الفاً وسبعمائة وثمان وسبعون ريال، نطلب من فضيلتكم إلزام ا

In [10]:
def show_random_elements(dataset, num_examples=5):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, datasets.ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
    display(HTML(df.to_html()))

In [11]:
show_random_elements(raw_datasets["train"])

In [12]:
metric

EvaluationModule(name: "rouge", module_type: "metric", features: [{'predictions': Value(dtype='string', id='sequence'), 'references': Sequence(feature=Value(dtype='string', id='sequence'), length=-1, id=None)}, {'predictions': Value(dtype='string', id='sequence'), 'references': Value(dtype='string', id='sequence')}], usage: """
Calculates average rouge scores for a list of hypotheses and references
Args:
    predictions: list of predictions to score. Each prediction
        should be a string with tokens separated by spaces.
    references: list of reference for each prediction. Each
        reference should be a string with tokens separated by spaces.
    rouge_types: A list of rouge types to calculate.
        Valid names:
        `"rouge{n}"` (e.g. `"rouge1"`, `"rouge2"`) where: {n} is the n-gram based scoring,
        `"rougeL"`: Longest common subsequence based scoring.
        `"rougeLsum"`: rougeLsum splits text using `"
"`.
        See details in https://github.com/huggingface/

## Preprocessing the data

In [14]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

tokenizer_config.json:   0%|          | 0.00/335 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/2.44M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


## Test the tokenizer and preprocess_function frunction

In [15]:
if model_checkpoint in ["malmarjeh/t5-arabic-text-summarization"]:
    prefix = "summarize: "
else:
    prefix = ""

In [16]:
max_input_length = 1024
max_target_length = 512

def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    labels = tokenizer(text_target=examples["answer"], max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [17]:
preprocess_function(raw_datasets['train'][:2])

{'input_ids': [[35880, 4633, 16410, 33, 237, 227, 2347, 2489, 15, 448, 18, 718, 70, 33, 196, 48, 2559, 5060, 2321, 1001, 59453, 40062, 2, 15, 1686, 872, 5841, 72616, 4621, 11115, 43761, 3, 13, 96779, 1451, 11046, 28, 6, 209, 544, 3, 293, 4, 1979, 4, 48, 4, 16, 1707, 15307, 52428, 40325, 209, 544, 200, 4, 68, 4, 16, 4, 174, 4, 174, 4, 44, 4, 1271, 133, 52428, 5149, 1834, 43, 420, 43, 25793, 6413, 48, 26151, 4948, 43, 102, 43, 88, 71428, 2248, 26151, 5393, 43, 420, 43, 1848, 80132, 20, 222, 26, 98139, 29, 98383, 20434, 89, 1686, 21667, 6373, 18883, 175, 1485, 1122, 49, 1462, 2321, 4324, 18885, 209, 1798, 33, 1087, 32547, 4247, 18, 6091, 10319, 872, 33, 52428, 9339, 1798, 872, 33, 26, 9286, 47887, 62194, 23272, 29, 43, 52761, 14707, 409, 44643, 56, 296, 7912, 2, 175, 326, 49194, 226, 3472, 15, 14155, 3714, 2449, 3751, 44643, 56, 296, 6, 3383, 182, 24515, 47, 49, 182, 23768, 47, 328, 1917, 6206, 1346, 360, 49194, 226, 765, 44643, 56, 296, 3231, 49538, 26, 5888, 4, 47306, 5888, 4, 38459, 13

## Apply the tokenizer function on our dataset

In [18]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

Map:   0%|          | 0/24 [00:00<?, ? examples/s]

Map:   0%|          | 0/7 [00:00<?, ? examples/s]

## Fine-tuning the model

In [19]:
config = AutoConfig.from_pretrained(model_checkpoint)

In [20]:
model = AutoModelForSeq2SeqLM.from_pretrained(
    model_checkpoint, config=config
)

pytorch_model.bin:   0%|          | 0.00/1.13G [00:00<?, ?B/s]

In [21]:
batch_size = 16
model_name = model_checkpoint.split("/")[-1]
args = Seq2SeqTrainingArguments(
    f"{model_name}-finetuned-xsum",
    evaluation_strategy="epoch",
    save_total_limit=3,
    num_train_epochs=50,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    predict_with_generate=True,
    fp16=False,
    push_to_hub=False,          # Disable mixed precision training
    fp16_full_eval=False   # Disable mixed precision evaluation
)


In [22]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [23]:

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    # Note that other metrics may not have a `use_aggregator` parameter
    # and thus will return a list, computing a metric for each sentence.
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True, use_aggregator=True)
    # Extract a few results
    result = {key: value * 100 for key, value in result.items()}

    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}



In [ ]:
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

`train` method:

In [ ]:
trainer.train()

Save the model:

In [28]:
new_model_path = "/content/drive/MyDrive/Data/t5_arabic3"

In [ ]:
trainer.save_model(f"{new_model_path}/best")
# tokenizer.add_tokens(custom_labels)
tokenizer.save_pretrained(f"{new_model_path}/best")

Data that contains cases

In [29]:
d = pd.read_csv("/content/drive/MyDrive/dataset/dataset_v3.csv")

In [30]:
d.columns

Index(['Unnamed: 0.1', 'F1', 'court', 'city', 'court_id', 'court_date',
       'appeal_court', 'region', 'appeal_id', 'appeal_date', 'judgment_text',
       'appeal_text', 'Unnamed: 0', 'links', 'appeal_data', 'appleal_text',
       'judgment_regex', 'cases_text', 'text_for_simi'],
      dtype='object')

In [31]:
# Load the fine-tuned model
model_cp = f"{new_model_path}/best"
tokenizer = AutoTokenizer.from_pretrained(model_cp)
model = AutoModelForSeq2SeqLM.from_pretrained(model_cp)

# input text
input_text = d['cases_text'][20]

# Tokenize
input_ids = tokenizer.encode(input_text, return_tensors="pt")

# output
output_ids = model.generate(input_ids,max_length=150)
output_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)


In [33]:
d['cases_text'][20]

'على رسول ﷲ أما بعد: فلدى الدائرة التجارية الخامسة عشر وبناء على القضية رقم ٦٨١٧ لعام ١٤٤٢ ه المقامة تجاري (...) شركة ذلك (...) القاضي إبراهيم القصيبي (الوقائع) تتلخص وقائع الدعوى بالقدر اللازم للحكم فيها، انه بتاريخ ١٤٤٢/١٠/١٥ه تقدم المدعي وكالة محمد جمال الصيدلاني سجل (...) بصحيفة دعوى الى المحكمة التجارية بالرياض، ذكر فيها ان موكلته قامت بتوريد مواد البناء للمدعى عليها بثمن آجل، و نتج عن هذا التعامل مديونية بذمة المدعى عليها بمبلغ و قدره (٢٨١,٥٤٧.٦٠) ويطلب الزامها بدفعه، وبإحالة القضية الى الدائرة باشرت نظرها في جلستين. ففي جلسة ١٤٤٢/١٠/٢٢ه حضر المدعي وكالة ولحضوره حضرت افراح بنت بن ناصر الشهري سجل (...)، ولتحقق الدائرة بما ورد في المادة ٩٠ من اللائحة التنفيذية لنظام المحاكم التجارية فقد أفهمت الدائرة الأطراف أن هذه الدعوى تقع في ولاية اختصاصها القضائي بناء على الفقرة (١) من المادة ١٦ من نظام المحاكم التجارية كما رأت الدائرة أن هذه الدعوى مقبولة شكلا وتحققت فيها شروط قبول الدعوى ثم عرضت الدائرة الصلح على الأطراف وانتهاء الدعوى وديا فطلبا مهلة لذلك، ثم سألت الدائرة وكيل المدعية عن طل

In [32]:
print("Generated Summary:\n", output_summary)

Generated Summary:
 تتلخص القضية رقم ٦٩١٧ لعام ١٤٤٢ ه، بإلزام شركة اتحاد المقاولين بسداد مبلغ قدره ٢٧١,٥٤٧.٦٠ قيمة مواد بناء. ل " الدائرة التجارية " بإلزام شركة شركة شركة، بسداد مبلغ قدره ٢٧١,٥٤٧.٦٠ قيمة مواد بناء.


# TEST

In [ ]:
new_model_path = "/content/drive/MyDrive/Data/t5_arabic3"

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from IPython.display import HTML, display
import ipywidgets as widgets
from ipywidgets import Layout
from IPython.display import clear_output

# Load the fine-tuned model
model_cp = f"{new_model_path}/best"
tokenizer = AutoTokenizer.from_pretrained(model_cp)
model = AutoModelForSeq2SeqLM.from_pretrained(model_cp)

# Textarea widget
tx = widgets.Textarea(
    value='',
    placeholder='أكتب قضيتك هنا...',
    description='قضية جديده:',
    layout=Layout(height='150px', width='70%'),
    disabled=False
)

# Button
action_button = widgets.Button(description="لــخصها !")

# area
output = widgets.Output()

# Display output
def print_summarizations(btn):
    input_text = tx.value

    # Tokenize the input text
    input_ids = tokenizer.encode(input_text, return_tensors="pt")

    output_ids = model.generate(input_ids, max_length=300)
    output_summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    # Display the output
    with output:
        clear_output()
        display(HTML(f"<h4>الملخص:</h4><p>{output_summary}</p>"))


action_button.on_click(print_summarizations)
box = widgets.VBox([tx, action_button, output], layout=Layout(height='500px'))



## Judicial Assistant

In [ ]:
# Display the box
display(box)